In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev7
cdv, version 1.1.2
Python 3.10.7


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000
game_amount = (play_amount * 2) + 1 # singleton odd mojos

In [22]:
# from setup-sim notebook
# alice 4070016934
# bob 861183802
alice_sk = PrivateKey.from_bytes(bytes.fromhex("67959bf0ade73bc2aa69cab59d0aa1fded890696f178393f7740b330a808f7c6"))
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()
bob_pk = G1Element.from_bytes(bytes.fromhex("b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a"))
bob_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])
player_two_info = Program.to([bob_pk, bob_puzzle_hash])

In [4]:
%%bash
cdv rpc coinrecords --by puzzlehash "7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666"
cdv rpc coinrecords --by puzzlehash "e8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4"

[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665217886
    }
]
[
    {
        "coin": {
            "amount": 1000000000000,
            "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
            "puzzle_hash": "0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665217886
    }
]


# Create Game Coin
> Two waiting room coins available from Alice and Bob

[waiting-room.clsp](../code/waiting-room.clsp)
```clojure
(mod (
        PLAYER_PH
        PLAYER_PK 
        P1_PK
        P2_PK 
        RETURN_AMOUNT
        GAME_AMOUNT
        launcher_id
        singleton_full_puzzle_hash
    )
```

In [5]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash, 
    alice_pk,
    alice_pk, 
    bob_pk,
    play_amount,
    game_amount
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch10cy0fk9zs4slnlxvnq79gjdw8yfapdc82c7a0k6alljmrlyhcenqwwvzrt'

In [6]:
## bob
bob_waiting_room_puzzle = waiting_room_puzzle.curry(
    bob_puzzle_hash, 
    bob_pk,
    alice_pk, 
    bob_pk,
    play_amount,
    game_amount
)
bob_waiting_room_puzzle_hash = bob_waiting_room_puzzle.get_tree_hash()
bob_waiting_room_address = encode_puzzle_hash(bob_waiting_room_puzzle_hash, "txch")
bob_waiting_room_address

'txch1ar2fme6md4wysj87ygxy0dhahp7ym3eww4ejtjfm0f5wre99mx6qu7quv6'

## Terminate Puzzle (Curried)
[terminate-game.clsp](../code/terminate-game.clsp)
```clojure
(mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
```

In [8]:
player_fee = 50_000_000
p2_amount = play_amount - player_fee
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice_puzzle_hash,
    bob_puzzle_hash,
    p2_amount
)

## Coin Puzzle (Curried)
[coin.clsp](../code/coin.clsp)
```clojure
(mod (MOD TERMINATE_PUZZLE PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
```

In [9]:
def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

## Launcher Coin

In [10]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = "mainnet"
genesis_challenge = config["farmer"]["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_coin = coin_records[0].coin
print(f'alice_waiting_room_coin:\n {alice_waiting_room_coin.to_json_dict()}')
print()
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(bob_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

bob_waiting_room_coin = coin_records[0].coin
print(f'bob_waiting_room_coin:\n {bob_waiting_room_coin.to_json_dict()}')

alice_waiting_room_coin:
 {'parent_coin_info': '0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b', 'puzzle_hash': '0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666', 'amount': 1000000000001}

bob_waiting_room_coin:
 {'parent_coin_info': '0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97', 'puzzle_hash': '0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4', 'amount': 1000000000000}


In [11]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_waiting_room_coin.name(), # waiting_room's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)

launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id.hex()}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)
launcher_coin


launcher id: bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24


Coin { parent_coin_info: 72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867, puzzle_hash: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9, amount: 2000000000001 }

### Singleton Launcher
[singleton_top_layer.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer.clvm)

[singleton_launcher.clvm](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_launcher.clvm)
```clojure
(mod (singleton_full_puzzle_hash amount key_value_list)
```

In [12]:
launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic-tac-toe"), 
            ("p1_pk", alice_pk),
            ("p2_pk", bob_pk)
        ]
    ]
)
print_program(launcher_solution)

launcher_announcement = launcher_solution.get_tree_hash()
print(f'\nlauncher_announcement:\n{launcher_announcement}')

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())

(0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))

launcher_announcement:
94e1d162edcc65e02767b4a1907fa370bccaae65133ae5f62b1da8a7122bf044
{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0x72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff80808080

In [13]:
%%bash
opd ff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080
opd ffa0d878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffcff8601d1a94a2001ffffff8467616d658b7469632d7461632d746f65ffff8570315f706bb096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff8570325f706bb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a8080

(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))
(0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))


In [14]:
%%bash
brun '(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))' '(0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))'

((51 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001) (60 0x94e1d162edcc65e02767b4a1907fa370bccaae65133ae5f62b1da8a7122bf044))


In [15]:
print(f'launcher_announcement:\n{launcher_announcement}')
print(f'\nlauncher_coin_id:\n{launcher_id.hex()}')

# announcementID == sha256(coinID + message).
from chia.util.hash import std_hash
launcher_coin_announcement = std_hash(launcher_id + launcher_announcement)
print(f'\nlauncher_coin_announcement:\n{launcher_coin_announcement}')

launcher_announcement:
94e1d162edcc65e02767b4a1907fa370bccaae65133ae5f62b1da8a7122bf044

launcher_coin_id:
bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24

launcher_coin_announcement:
af04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0


In [16]:
alice_waiting_room_coin_spend = CoinSpend(
    alice_waiting_room_coin,
    alice_waiting_room_puzzle,
    Program.to([
        launcher_id,
        singleton_puzzle.get_tree_hash()
    ])
)
print_json(alice_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
        "puzzle_hash": "0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80808080ffff02ff2effff04ff02ffff04ff0dff8080808080ffff

In [17]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80808080ffff02ff2effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff04ffff04ff18ffff04ffff0bff5fffff02ff16ffff04ff02ffff04ff81bfffff04ff2fffff04ff0bffff04ff17ff8080808080808080ff808080ffff04ffff04ff10ffff04ff0bffff04ff5fff80808080ffff02ffff03ffff09ff05ff0b80ffff01ff04ffff04ff12ffff04ff1affff04ff2fff80808080ff8080ff8080ff01808080ff018080ffff04ffff01a0d210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64affff04ffff018600e8d4a51000ffff04ffff018601d1a94a2001ff0180808080808080
opd ffa0bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24ffa0d878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc80

(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c 11 (c 5 ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 11 (c 23 ()))))))) ())) (c (c 16 (c 11 (c 95 ()))) (a (i (= 5 11) (q 4 (c 18 (c 26 (c 47 ()))) ()) ()) 1))) 1)) (c (q . 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb

In [18]:
%%bash
brun '(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c 11 (c 5 ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 11 (c 23 ()))))))) ())) (c (c 16 (c 11 (c 95 ()))) (a (i (= 5 11) (q 4 (c 18 (c 26 (c 47 ()))) ()) ()) 1))) 1)) (c (q . 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x00e8d4a51000) (c (q . 0x01d1a94a2001) 1)))))))' '(0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc)'

((61 0xaf04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0) (50 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24) (51 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001))


In [19]:
bob_waiting_room_coin_spend = CoinSpend(
    bob_waiting_room_coin,
    bob_waiting_room_puzzle,
    Program.to([
        launcher_id,
        singleton_puzzle.get_tree_hash()
    ])
)
print_json(bob_waiting_room_coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000000,
        "parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97",
        "puzzle_hash": "0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80808080ffff02ff2effff04ff02ffff04ff0dff8080808080ffff

In [20]:
%%bash
opd ff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80808080ffff02ff2effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff04ffff04ff18ffff04ffff0bff5fffff02ff16ffff04ff02ffff04ff81bfffff04ff2fffff04ff0bffff04ff17ff8080808080808080ff808080ffff04ffff04ff10ffff04ff0bffff04ff5fff80808080ffff02ffff03ffff09ff05ff0b80ffff01ff04ffff04ff12ffff04ff1affff04ff2fff80808080ff8080ff8080ff01808080ff018080ffff04ffff01a03ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657bffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64affff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64affff04ffff018600e8d4a51000ffff04ffff018601d1a94a2001ff0180808080808080
opd ffa0bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24ffa0d878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc80

(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c 11 (c 5 ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 11 (c 23 ()))))))) ())) (c (c 16 (c 11 (c 95 ()))) (a (i (= 5 11) (q 4 (c 18 (c 26 (c 47 ()))) ()) ()) 1))) 1)) (c (q . 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb

In [21]:
%%bash
brun '(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c 11 (c 5 ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 11 (c 23 ()))))))) ())) (c (c 16 (c 11 (c 95 ()))) (a (i (= 5 11) (q 4 (c 18 (c 26 (c 47 ()))) ()) ()) 1))) 1)) (c (q . 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x00e8d4a51000) (c (q . 0x01d1a94a2001) 1)))))))' '(0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc)'

((61 0xaf04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0) (50 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24))


In [26]:
from chia.types.spend_bundle import SpendBundle

message: bytes = launcher_id
sig_alice_spend: G2Element = AugSchemeMPL.sign(
    alice_sk,
    message
    + alice_waiting_room_coin.name()
    #+ bytes.fromhex(genesis_challenge),
    + bytes.fromhex("ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb")
)

sig_bob_spend: G2Element = AugSchemeMPL.sign(
    alice_sk,
    message
    + bob_waiting_room_coin.name()
    #+ bytes.fromhex(genesis_challenge),
    + bytes.fromhex("ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb")
)

spend_bundle = SpendBundle([
        launcher_coin_spend,
        alice_waiting_room_coin_spend,
        bob_waiting_room_coin_spend
    ], 
    AugSchemeMPL.aggregate([sig_alice_spend, sig_bob_spend])
)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0x9958918817d378f0436a0eadaa4fe24865487f5b80e9899aeb69de50b8e245fdbad85fba61c5a56903658022f38a0d4e11c241572a8413fcd15b2bb054cba1994b86b2c834ee7d96f373fb19103e45fce77dd4f4a17437be3eb183845970a34c",
    "coin_spends": [
        {
            "coin": {
                "amount": 2000000000001,
                "parent_coin_info": "0x72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867",
                "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
            },
            "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
            "solution": "0xffa0d878ddc7c62757e6116f7e843fee3eae2374062d1

In [27]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'SUCCESS', 'success': True}


In [29]:
%%bash
cdv sim -n tic-tac-toe farm

Farmed 1 Transaction blocks
Block Height is now: 9


In [36]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(singleton_puzzle.get_tree_hash())
full_node_client.close()
await full_node_client.await_closed()
print_json(coin_records[0].to_json_dict())
game_coin = coin_records[0].coin


{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24",
        "puzzle_hash": "0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc"
    },
    "coinbase": false,
    "confirmed_block_index": 7,
    "spent_block_index": 0,
    "timestamp": 1665219677
}


```sh
❯ cdv inspect spendbundles $spendbundle --debug --cost
[{"aggregated_signature": "0x9958918817d378f0436a0eadaa4fe24865487f5b80e9899aeb69de50b8e245fdbad85fba61c5a56903658022f38a0d4e11c241572a8413fcd15b2bb054cba1994b86b2c834ee7d96f373fb19103e45fce77dd4f4a17437be3eb183845970a34c", "coin_solutions": [{"coin": {"parent_coin_info": "0x72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867", "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9", "amount": 2000000000001}, "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080", "solution": "0xffa0d878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffcff8601d1a94a2001ffffff8467616d658b7469632d7461632d746f65ffff8570315f706bb096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff8570325f706bb0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a8080"}, {"coin": {"parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b", "puzzle_hash": "0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666", "amount": 1000000000001}, "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80808080ffff02ff2effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff04ffff04ff18ffff04ffff0bff5fffff02ff16ffff04ff02ffff04ff81bfffff04ff2fffff04ff0bffff04ff17ff8080808080808080ff808080ffff04ffff04ff10ffff04ff0bffff04ff5fff80808080ffff02ffff03ffff09ff05ff0b80ffff01ff04ffff04ff12ffff04ff1affff04ff2fff80808080ff8080ff8080ff01808080ff018080ffff04ffff01a0d210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64affff04ffff018600e8d4a51000ffff04ffff018601d1a94a2001ff0180808080808080", "solution": "0xffa0bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24ffa0d878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc80"}, {"coin": {"parent_coin_info": "0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97", "puzzle_hash": "0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4", "amount": 1000000000000}, "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff02ff3effff04ff02ffff04ff0bffff04ff17ffff04ff2fffff04ff81bfffff04ff82017fffff04ff8202ffff808080808080808080ffff01ff04ffff04ff14ffff04ff1cff808080ffff04ffff04ff12ffff04ff05ffff04ff5fff80808080ffff04ffff04ff10ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffffff323dff5264ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff02ff2effff04ff02ffff04ffff04ff05ffff04ff0bffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff1780ffff04ffff04ffff018570325f706bff2f80ff80808080ff80808080ff80808080ffff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff2effff04ff02ffff04ff09ff80808080ffff02ff2effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff04ffff04ff18ffff04ffff0bff5fffff02ff16ffff04ff02ffff04ff81bfffff04ff2fffff04ff0bffff04ff17ff8080808080808080ff808080ffff04ffff04ff10ffff04ff0bffff04ff5fff80808080ffff02ffff03ffff09ff05ff0b80ffff01ff04ffff04ff12ffff04ff1affff04ff2fff80808080ff8080ff8080ff01808080ff018080ffff04ffff01a03ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657bffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64affff04ffff01b096f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010ffff04ffff01b0b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64affff04ffff018600e8d4a51000ffff04ffff018601d1a94a2001ff0180808080808080", "solution": "0xffa0bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24ffa0d878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc80"}]}]
Cost: 33623004

Debugging Information
---------------------
================================================================================
consuming coin (0x72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001)
  with id b'\xbcS\x80Wk\xa1)F\x7f\xe7v\xcf\xeak\xc0\xb8j\xd6E6\x97\xf1\rQ6\x0c\xcdz\xc40<$'


brun -y main.sym '(a (q 4 (c 4 (c 5 (c 11 ()))) (c (c 10 (c (a 14 (c 2 (c (c 5 (c 11 (c 23 ()))) ()))) ())) ())) (c (q 51 60 2 (i (l 5) (q 11 (q . 2) (a 14 (c 2 (c 9 ()))) (a 14 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 1))' '(0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001 (("game" . "tic-tac-toe") ("p1_pk" . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) ("p2_pk" . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a)))'

((CREATE_COIN 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001) (CREATE_COIN_ANNOUNCEMENT 0x94e1d162edcc65e02767b4a1907fa370bccaae65133ae5f62b1da8a7122bf044))

grouped conditions:

  (CREATE_COIN 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001)

  (CREATE_COIN_ANNOUNCEMENT 0x94e1d162edcc65e02767b4a1907fa370bccaae65133ae5f62b1da8a7122bf044)


-------
consuming coin (0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b 0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666 0x00e8d4a51001)
  with id b'r\xa6\x14\xf2q\x99\x8c\x9d\xdd\x0b\xf7\x9b+e`\\e\x0bs>\xd3\xbas\x9f\xfe\xa4\x10\xddYv\xd8g'


brun -y main.sym '(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c 11 (c 5 ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 11 (c 23 ()))))))) ())) (c (c 16 (c 11 (c 95 ()))) (a (i (= 5 11) (q 4 (c 18 (c 26 (c 47 ()))) ()) ()) 1))) 1)) (c (q . 0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x00e8d4a51000) (c (q . 0x01d1a94a2001) 1)))))))' '(0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc)'

((ASSERT_COIN_ANNOUNCEMENT 0xaf04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0) (AGG_SIG_ME 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24) (CREATE_COIN 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001))

grouped conditions:

  (ASSERT_COIN_ANNOUNCEMENT 0xaf04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0)

  (AGG_SIG_ME 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24)

  (CREATE_COIN 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001)


-------
consuming coin (0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97 0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4 0x00e8d4a51000)
  with id b'U/V\x1c\x10:!J\x9bF\x19\xc7>J\xdc\xccy\xb4\xd6\xde\x03\x9c\x0c^\x0f\xf8\x81rG\xca\x14$'


brun -y main.sym '(a (q 2 (q 2 (i 383 (q 2 62 (c 2 (c 11 (c 23 (c 47 (c -65 (c 383 (c 767 ())))))))) (q 4 (c 20 (c 28 ())) (c (c 18 (c 5 (c 95 ()))) (c (c 16 (c 11 (c 5 ()))) ())))) 1) (c (q ((50 . 61) 82 . 100) (51 . 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9) (a 46 (c 2 (c (c 5 (c 11 (c (c (q "game" . "tic-tac-toe") (c (c (q . "p1_pk") 23) (c (c (q . "p2_pk") 47) ()))) ()))) ()))) (a (i (l 5) (q 11 (q . 2) (a 46 (c 2 (c 9 ()))) (a 46 (c 2 (c 13 ())))) (q 11 (q . 1) 5)) 1) 4 (c 24 (c (sha256 95 (a 22 (c 2 (c -65 (c 47 (c 11 (c 23 ()))))))) ())) (c (c 16 (c 11 (c 95 ()))) (a (i (= 5 11) (q 4 (c 18 (c 26 (c 47 ()))) ()) ()) 1))) 1)) (c (q . 0x3ac42b719d0b34879714d1e601721f08ba317862d2118fa16aa7e8bf1a19657b) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010) (c (q . 0xb1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a) (c (q . 0x00e8d4a51000) (c (q . 0x01d1a94a2001) 1)))))))' '(0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc)'

((ASSERT_COIN_ANNOUNCEMENT 0xaf04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0) (AGG_SIG_ME 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24))

grouped conditions:

  (ASSERT_COIN_ANNOUNCEMENT 0xaf04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0)

  (AGG_SIG_ME 0x96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010 0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24)


-------

spent coins
  (0xfc669e48c63de848a7159610118f21d96efeb328af1876afd91a17cf1fecda97 0xe8d49de75b6d5c4848fe220c47b6fdb87c4dc72e757325c93b7a68e1e4a5d9b4 0x00e8d4a51000)
      => spent coin id b'U/V\x1c\x10:!J\x9bF\x19\xc7>J\xdc\xccy\xb4\xd6\xde\x03\x9c\x0c^\x0f\xf8\x81rG\xca\x14$'
  (0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b 0x7e08f4d8a28561f9fccc983c5449ae3913d0b707563dd7db5dffe5b1fc97c666 0x00e8d4a51001)
      => spent coin id b'r\xa6\x14\xf2q\x99\x8c\x9d\xdd\x0b\xf7\x9b+e`\\e\x0bs>\xd3\xbas\x9f\xfe\xa4\x10\xddYv\xd8g'

created coins
  (0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24 0xd878ddc7c62757e6116f7e843fee3eae2374062d16b3372500fceba6e9adbffc 0x01d1a94a2001)
      => created coin id b'\nS\x17\xc3\xe8V\xd0o\xc1\xeb4\xc1Pt^$\x1b\xc8\x06Mt\xa9\xae5\xb9T\x10\xf0\xea|\x0eD'

ephemeral coins
  (0x72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867 0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9 0x01d1a94a2001)
      => created coin id b'\xbcS\x80Wk\xa1)F\x7f\xe7v\xcf\xeak\xc0\xb8j\xd6E6\x97\xf1\rQ6\x0c\xcdz\xc40<$'
created coin announcements
  ['0xbc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24', '0x94e1d162edcc65e02767b4a1907fa370bccaae65133ae5f62b1da8a7122bf044'] =>
      af04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0


zero_coin_set = []

created  coin announcements = ['af04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0']

asserted coin announcements = ['af04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0', 'af04a998f8a00160a5d04b41759f80a3bdad932fc6d7d9c08dbce97701551be0']

symdiff of coin announcements = []


================================================================================

aggregated signature check pass: True
pks: [<G1Element 96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010>, <G1Element 96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010>]
msgs: ['bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c2472a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb', 'bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24552f561c103a214a9b4619c73e4adccc79b4d6de039c0c5e0ff8817247ca1424ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb']
  msg_data: ['bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24', 'bc5380576ba129467fe776cfea6bc0b86ad6453697f10d51360ccd7ac4303c24']
  coin_ids: ['72a614f271998c9ddd0bf79b2b65605c650b733ed3ba739ffea410dd5976d867', '552f561c103a214a9b4619c73e4adccc79b4d6de039c0c5e0ff8817247ca1424']
  add_data: ['ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb', 'ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb']
signature: 9958918817d378f0436a0eadaa4fe24865487f5b80e9899aeb69de50b8e245fdbad85fba61c5a56903658022f38a0d4e11c241572a8413fcd15b2bb054cba1994b86b2c834ee7d96f373fb19103e45fce77dd4f4a17437be3eb183845970a34c
None
```